# Automating insurance claim processing using BDA and Bedrock Agents


## Introduction

Generative AI agents are a powerful tool for large enterprises, as they can enhance operational efficiency, customer service, and decision-making while reducing costs and enabling innovation. These agents excel at automating a wide range of routine and repetitive tasks, and can orchestrate complex, multi-step workflows.

Amazon Bedrock Agents and Amazon Bedrock Knowledge Bases enable the creation of specialized agents that can automate tasks associated with the enterprise workflows, efficiently scale and improve customer service, and enhance decision support through improved knowledge management.

Integrating Amazon Bedrock Data Automation (BDA) with the Bedrock Agents and Knowledge Bases provides a comprehensive, automated, and scalable solution for efficiently processing and extracting insights from diverse data sources for your organization's use cases.

In this notebook, we go through an end-to-end solution for one such use case, Insurance claims management. Insurance companies deal with a high volume of claims, which can be time-consuming and prone to errors when processed manually. By leveraging capabilities provided by Amazon Bedrock including Bedrock Data Automation and Bedrock Agents, We will explore a generative AI-powered solution that streamlines the claim process, improves efficiency, and enhances customer experience. 

There is an associated deployable solution in this [Github repository](https://github.com/aws-solutions-library-samples/guidance-for-multimodal-data-processing-using-amazon-bedrock-data-automation).

## Architecture

----
The architecture for the solution is shown below. The process begins with a Medical Claim Form (CMS 1500) received from a medical provider -

### Claim Review Process

- **Step 1:** Medical provider submits the claim, which is stored in an S3 bucket.

- **Step 2:** An S3 Object Created event is matched by a configured EventBridge Rule.
- **Step 3:** EventBridge rule triggers a AWS Lambda function.
- **Step 4:** The Lambda function in turn calls the `InvokeDataAutomationAsync` job in BDA along with a custom blueprint.
- **Step 5:** BDA uses the provided custom blueprint for the CMS 1500 form to extract the content of the -claim.BDA stores extracted claims data in the S3 bucket specified in the API call.
- **Step 6:** BDA sends a Job completion event to EventBridge that include the job status and the S3 uri of the response and metadata.
- **Step 7:** EventBridge rule triggers an AWS Lambda function
- **Step 8:** The Lambda function uses the BDA job response and metadata to fetch extracted claim form data.
- **Step 9:** The Lambda function then invokes the pre-configured Bedrock Agent to process the claim
- **Step 10:** Bedrock Agent uses configured agent actions (fulfilled by Lambda function) to perform tasks required during claim verification process.
- **Step 11:** Bedrock Agent uses agent action, implemented using Lambda function, to query member and patient information stored in Aurora Postgres in order to complete claim verification. Agent also uses the agent action  to stores verified claim details in the database.
- **Step 12:** Bedrock Agent query Bedrock knowledge Base to gathers coverage details on the treatments, services and supplies provided in the claims form
- **Step 13:** Bedrock Agent create a final report to detail the validation process and stores the report in S3


### Claim Policy Knowledge Ingestion Process
The Knowledge Base is used for Retrieval Augmented Generation (RAG), a technique that uses information from data sources to improve the relevancy and accuracy of generated responses. In this example, the knowledge base converts claim policy documents in pdf format into vector embeddings and stored in a vector store (Amazon Opensearch serverless vector index). This process of converting the data into vector embeddings is called ingestion. The Ingestion process is carried out to make the knowledge base ready to be queried by the agent (or humans). 

[More details on the Ingestion process](https://docs.aws.amazon.com/bedrock/latest/userguide/kb-how-data.html)

- **Step 1:** Medical provider uploads claim policy document to S3 bucket.
  
- **Step 2:** An S3 Object Created event is matched by a configured EventBridge Rule.
- **Step 3:** EventBridge rule triggers a AWS Lambda function.
- **Step 4:** Lambda function in turn invokes `StartIngestionJob` api to trigger a Knowledge Base datasource sync job
- **Step 5:** The datasource sync job incrementally converts the raw claim policy documents in the S3 bucket into vector embeddings, based on the vector embeddings model and configurations specified 
- **Step 6:** The vector embeddings are store in the configured vector store (opensearch serverless in this case)

![Claims Review Architecture](data/images/Medical_Claims_Processing_Architecture.png)

## Prerequisites
Before starting this notebook, ensure you have:

1. An AWS account with access to Amazon Bedrock
2. Necessary IAM permissions to create and manage Bedrock resources
3. <a id="stack">Install the Insurance Claims Review stack using AWS CloudFormation with this</a> [template](https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/c64e3606-ab68-4521-81ea-b2eb36c993b9/templates/bda-idp-workshop.yaml).

## Setup
In the following sections we would run through the process to setup the AWS resources required to run the end-to-end flow for claim processing

### Install Required Libraries
First, let's import the necessary libraries.

In [ ]:
%pip install "boto3>=1.37.6" itables==2.2.4 PyPDF2==3.0.1 --upgrade -q

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import boto3
import json
from utils import helper_functions
from utils import bedrock_utils
from utils import display_functions
import pandas as pd
import uuid
from IPython.display import JSON, display, IFrame, Markdown
import os
import shutil
from urllib.parse import urlparse
import sagemaker


sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()['Account']
region_name = boto3.session.Session().region_name


# Set up AWS credentials (make sure you have the appropriate permissions)
session = boto3.Session()
sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()

iam = boto3.client('iam')
s3_client = session.client('s3')
bedrock = session.client('bedrock')
bedrock_agent = session.client('bedrock-agent')
bedrock_agent_runtime = session.client('bedrock-agent-runtime')
bda_client = boto3.client('bedrock-data-automation')
bda_runtime_client = boto3.client('bedrock-data-automation-runtime')

### Set BDA Input / Output Locations in S3
BDA `InvokeDataAutomationAsync` API uses S3 for receiving input files to process and to store the output results. We set a S3 location each for input files and output results.

In [ ]:
bda_workshop_s3_location = f's3://{default_bucket}/bda-workshop'
bda_s3_input_location = f'{bda_workshop_s3_location}/input'
bda_s3_output_location = f'{bda_workshop_s3_location}/output'
agent_review_s3_input_location = f'{bda_workshop_s3_location}/agent_input'
eoc_document_s3_location = f'{bda_workshop_s3_location}/eoc'

## Creating and Configuring Bedrock Knowledge Base
Amazon Bedrock Knowledge Bases help you take advantage of Retrieval Augmented Generation (RAG), a popular technique that involves drawing information from a data store to augment the responses generated by Large Language Models (LLMs). When you set up a knowledge base with your data source, your application can query the knowledge base to return information to answer the query either with direct quotations from sources or with natural responses generated from the query results.

In our solution, we use Amazon Bedrock Knowledge Base for enriching our claim processing with coverage and policy information from the evidence of coverage documents that we ingest into the Knowledge Base through our data source.

### Fetch attributes for pre-configured resources
Many of AWS resources that are used in our solution would be pre-created so that we can focus on the BDA and Bedrock Agents part of the solution. See [Prerequisites](#stack)

Here we extract some of the resource attributes from our pre-created stack. that our Bedrock Knowlege Base resources would need to use.

In [ ]:
stack_resource_attributes = helper_functions.get_stack_outputs()
embedding_model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v2:0'
kb_role_arn = stack_resource_attributes['KBServiceRole']
vector_store_index_name = stack_resource_attributes['ClaimsVectorStoreIndexName']
vector_store_collection_arn = stack_resource_attributes['ClaimsVectorStoreCollectionArn']
vector_store_collection_name = stack_resource_attributes['ClaimsVectorStoreCollectionName']

### Create the Knowledge Base
A Knowledge Base for evidence of coverage (EoC) documents in insurance claim processing serves as a centralized repository that stores and organizes policy documentation, coverage details, and related insurance terms. It enables the automated system to quickly access, interpret, and validate coverage information when processing claims by comparing incoming claims against stored policy rules and conditions. This streamlines verification of coverage eligibility, policy limits, exclusions, and specific terms, reducing manual lookup time and potential human errors while ensuring consistent and accurate claim decisions based on the documented coverage parameters.

Here we create a Bedrock Knowledge Base with an OpenSearch Serverless Vector store (pre-created) and an S3 datasource where we upload our EoC documents.

In [ ]:
knowledge_base_id, status = bedrock_utils.create_knowledge_base(
    bedrock_agent,
    kb_name=f'claims-eoc-kb-{account_id}',
    kb_description='Knowledge Base for Insurance Evidence of Coverage documents that details the plan\'s costs and benefits',
    embedding_model_arn=embedding_model_arn,
    kb_role_arn=kb_role_arn,
    vector_store_collection_arn=vector_store_collection_arn,
    vector_store_index_name=vector_store_index_name)

### Connecting a data source to the knowledge base
After finishing the configurations for our knowledge base, we connect a supported data source to the knowledge base. Data sources contain information returned when querying a Knowledge Base. For our solution, we connect to a custom data source. Using a custom data source provides use the ability to use the `KnowledgeBaseDocuments` API operations to directly ingest or delete documents without the need to sync changes. For details see [Connect your knowledge base to a custom data source](https://docs.aws.amazon.com/bedrock/latest/userguide/custom-data-source-connector.html)

In [ ]:
data_source_id, status = bedrock_utils.create_data_source(bedrock_agent, knowledge_base_id, datasource_name=f'claims-eoc-datasource-{account_id}')

### Ingesting Documents into Knowledge Base
To demonstrate the concept of using Knowledge bases to automatically enrich claim processing lifecycle with coverage information, We use a set of machine-generated EoC documents and ingest them directly to our Knowledge Base we created earlier.

#### Upload EoC Documents to S3

In [ ]:
document_urls = [
    'https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/c64e3606-ab68-4521-81ea-b2eb36c993b9/samples/Evidence_of_Coverage_-_AnyHealth_Premium.pdf',
    'https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/c64e3606-ab68-4521-81ea-b2eb36c993b9/samples/Evidence_of_Coverage_-_AnyHealth_Standard.pdf',
    'https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/c64e3606-ab68-4521-81ea-b2eb36c993b9/samples/Evidence_of_Coverage_-_AnyHealth_Plus.pdf']

local_documents_path = 'data/documents'

eoc_documents_path = os.path.join(local_documents_path, 'eoc')
# Create full path of directories
os.makedirs(eoc_documents_path, exist_ok=True)
for document_url in document_urls:
    parsed = urlparse(document_url)
    local_file_name = parsed.path.split('/')[-1]
    local_file_path = os.path.join(eoc_documents_path, local_file_name)
    !curl {document_url} --output {local_file_path}

!aws s3 cp {eoc_documents_path} {eoc_document_s3_location} --recursive


#### Ingest EoC Documents directly into our Knowledge Base

After setting up our custom data source, we can add our EoC documents into it and directly ingest them into the knowledge base. Unlike other data sources, we don't need to sync a custom data source. This feature allows us to modify our data source and sync the changes in one step. For details see [Ingest changes directly into a knowledge base](https://docs.aws.amazon.com/bedrock/latest/userguide/kb-direct-ingestion.html).

When ingesting documents directly into Knowledge Base, we need to provide the `DocumentContent` to the `IngestKnowledgeBaseDocuments` API, that contains information about a document to ingest into a knowledge base and any metadata to associate with it.

In [ ]:
documents = [{
    'plan_name':'AnyHealth_Standard',
    'document_id': 'Evidence_of_Coverage_-_AnyHealth_Standard.pdf',
    'document_uri': f'{eoc_document_s3_location}/Evidence_of_Coverage_-_AnyHealth_Standard.pdf'
},
{
    'plan_name':'AnyHealth_Premium',
    'document_id': 'Evidence_of_Coverage_-_AnyHealth_Premium.pdf',
    'document_uri': f'{eoc_document_s3_location}/Evidence_of_Coverage_-_AnyHealth_Premium.pdf'
},
{
    'plan_name':'AnyHealth_Plus',
    'document_id': 'Evidence_of_Coverage_-_AnyHealth_Plus.pdf',
    'document_uri': f'{eoc_document_s3_location}/Evidence_of_Coverage_-_AnyHealth_Plus.pdf'
}]


In [ ]:
results = bedrock_utils.ingest_and_wait(bedrock_agent, data_source_id, knowledge_base_id, documents)
results_view = [(item['identifier']['custom']['id'], item['status']) for item in results]
display(pd.DataFrame(results_view).style.hide(axis='index').hide(axis='columns'))

## Creating an Agent
Amazon Bedrock Agents offers you the ability to build and configure autonomous agents your applications. AI Agents can interact with users, make decisions, and take actions. These agents combine foundation models with organizational data, APIs, and knowledge bases to handle complex tasks and conversations, simplifying the development of generative AI applications by providing a pre-built framework for integrating various AI capabilities and data sources.

To create an agent with Amazon Bedrock, you set up the following components:

<table width="80%">
    <tr>
        <th width="30%"  align="left" style="background-color: #999693">Configuration</th>
        <th width="70%" align="left" style="background-color: #999693">Description</th>
    </tr>
    <tr>
        <td><b>Agent resource role</b></td>
        <td>A service role with permissions to call API operations on the agent</td>
    </tr>
    <tr>
        <td><b>Foundation Model</b></td>
        <td>An FM for the agent to invoke to perform orchestration</td>
    </tr>
    <tr>
        <td><b>Instructions</b></td>
        <td>You write instructions that describe what the agent is designed to do. With advanced prompts, you can further customize instructions for the agent at every step of orchestration and include Lambda functions to parse each step's output</td>
    </tr>
    <tr>
        <td><b>Action Group</b></td>
        <td>Defines actions that the agent can help end users perform. Each action group includes the parameters that the agent must provide when invoking the action.<br><br>
        See <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html">Use action groups to define actions for your agent</a>
        </td>
    </tr>
    <tr>
        <td><b>Knowledge Base</b></td>
        <td>Provides a repository of information that the agent can query to answer customer queries and improve its generated responses
            <br><br><a href="https://docs.aws.amazon.com/bedrock/latest/userguide/agents-kb-add.html">Augment response generation for your agent with knowledge base</a>
        </td>
    </tr>

</table>


For our solution, we create an agent that helps customers process insurance claims. 

### Fetch attributes for pre-configured resources
Many of AWS resources that are used in our solution would be pre-created so that we can focus on the BDA and Bedrock Agents part of the solution. See [Prerequisites](#stack)

Here we extract some of the resource attributes from our pre-created stack that the Bedrock Agent resources would need to use.

### Choosing a foundation model
You choose a foundation model (FM) that the agent invokes to interpret user input and subsequent prompts in its orchestration process. The agent also invokes the FM to generate responses and follow-up steps in its process.

**When choosing a model please follow the model provider acceptable end user policy**.

<details>
  <summary>More Details </summary>

        
- [Model support by feature](https://docs.aws.amazon.com/bedrock/latest/userguide/models-features.html) 
    
- [Model support by AWS Region in Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/models-regions.html)

<div style="background-color: #f0f7fb; padding: 15px; border-left: 6px solid #2196F3; margin-bottom: 15px;">
<span style="color: #2196F3;">ⓘ Note</span><br>
Some models are accessible in some Regions only through cross-region inference. To learn more about cross-region inference, see 
    <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html">Increase throughput with cross-region-inference<a> and <a href="https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles-support.html">Supported Regions and models for inference profiles</a>.
</div>


In [ ]:
#Change this to the model id for your preferred Foundation Model for Amazon Bedrock Agent
foundation_model_id = 'us.anthropic.claude-3-5-haiku-20241022-v1:0' 

stack_resource_attributes = helper_functions.get_stack_outputs()
agent_service_role_arn = stack_resource_attributes['AgentServiceRole']
agent_actions_lambda_arn = stack_resource_attributes['ClaimsReviewAgentActionLambdaFunction']

### Setup Agent Instruction
We have a predesigned agent instruction that we will use for our claims processing agent. The instruction  describe what the agent is designed to do. We use base prompt templates for the Bedrock Agent for the purpose of this guidance solution. 

You can enhance your agent's accuracy through modifying these prompt templates to provide detailed configurations. See [Enhance agent's accuracy using advanced prompt templates in Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/advanced-prompts.html)

In [ ]:
with open('data/agent_resources/agent_prompt.txt','r') as f:
    agent_instruction = f.read()
helper_functions.show_popup_link('View Agent Instruction', agent_instruction, "agent_instruction")

### Create Agent
With the Agent Instruction setup and foundation model chosen, we can now create our agent that can help with insurance claims processing. Subsequently we would define an action group and setup a Knowledge Base for the agent.

In [ ]:
agent_id, status, version, agent_arn = bedrock_utils.create_agent(bedrock_agent,
    agentName=f'claims-agent-{account_id}',
    agent_service_role_arn=agent_service_role_arn,
    description='claims review agent',
    foundation_model_id=foundation_model_id,
    agent_instruction=agent_instruction,
    orchestrationType='DEFAULT'
)
version = version or 'DRAFT'

### Create Agent Action Group
We use Action groups with our Bedrock agent to define the actions that the agent should perform. We setup an Open API Schema with descriptions, structure, and parameters that define each action in the action group as an API operation.

The agent uses the schema to determine the API operation that it needs to invoke and the parameters that are required to make the API request. These details are then sent through to the pre-created Lambda function that has the business logic to carry out the action.

In [ ]:
with open('data/agent_resources/agent_action_schema.json') as f:
    api_schema = f.read()
helper_functions.show_popup_link('View OpenAPI Schema for Actions', api_schema, "action_schema")

In [ ]:
create_action_group_response = bedrock_utils.create_agent_action_group(bedrock_agent,
    actionGroupName='claims-review-actions',
    description='claims review agent actions',
    actionGroupState='ENABLED',
    agentId=agent_id,
    agentVersion=version,
    apiSchema={
        'payload': api_schema,
    },
    agent_actions_lambda_arn=agent_actions_lambda_arn
)

In [ ]:
# Example usage
agent_action_lambda_function_arn = stack_resource_attributes['ClaimsReviewAgentActionLambdaFunction']
bedrock_utils.add_lambda_permission(
    function_name=agent_action_lambda_function_arn.split(":")[-1],
    principal='bedrock.amazonaws.com',
    action='lambda:InvokeFunction',
    source_arn=agent_arn
)

In [ ]:
associate_agent_kb_response = bedrock_utils.associate_agent_knowledge_base(bedrock_agent,
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Claims Evidence of Coverage, used to verify if claims are covered under specific coverage plan terms',
    knowledgeBaseId=knowledge_base_id,
    knowledgeBaseState='ENABLED'
)

In [ ]:
bedrock_agent.prepare_agent(agentId=agent_id)
status_response = helper_functions.wait_for_completion(
                client=bedrock_agent,
                get_status_function=bedrock_agent.get_agent,
                status_kwargs={
                    'agentId': agent_id
                },
                completion_states=['PREPARED'],
                error_states=['FAILED'],
                status_path_in_response='agent.agentStatus',
                max_iterations=5,
                delay=15
    )

In [ ]:
agent_alias_id, status  = bedrock_utils.create_agent_alias(bedrock_agent,
    agentAliasName='LIVE',
    agentId=agent_id,
    description='LIVE version of agent'
)

## Setting up Bedrock Data Automation

### Sample Document

For this lab, we use a sample `Medical Claim` pack. The pack contains the medical claims form (CMS 1500) as well as supporting documents including identify documents etc. For the purpose of this solution, we would be focussing on the CMS 1500 form to extract claims data using a custom blueprint with BDA.

### Download sample and store to S3 Input location

Let's download the claims pack sample and store it in the S3 Location that we use for BDA input

In [ ]:
# Download the document
document_url = 'https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/c64e3606-ab68-4521-81ea-b2eb36c993b9/samples/sample1_cms-1500-P.pdf'
local_documents_path = 'data/documents'

# Create full path of directories
os.makedirs(local_documents_path, exist_ok=True)
local_file_name = 'sample1_cms-1500-P.pdf'
local_file_path = os.path.join(local_documents_path, local_file_name)
!curl {document_url} --output {local_file_path}

document_s3_uri = f'{bda_s3_input_location}/{local_file_name}'
target_s3_bucket, target_s3_key = helper_functions.get_bucket_and_key(document_s3_uri)
s3_client.upload_file(local_file_path, target_s3_bucket, target_s3_key)

print(f"Downloaded file to: {local_file_path}")
print(f"Uploaded file to S3: {target_s3_key}")
print(f"document_s3_uri: {document_s3_uri}")

### View Sample Document

In [ ]:
IFrame(local_file_path, width=800, height=600)

### Create Blueprint for Medical Claim form (CMS 1500)

Our sample file contains multiple document types, but we are only interested in the data in the Claims form. So we use a single custom blueprints to process the document. 

We use the create_blueprint operation (or update_blueprint to update an existing blueprint) in the boto3 API to create/update the blueprint. Each blueprint that you create is an AWS resource with its own blueprint ID and ARN.

In [ ]:
blueprint = {
    "name": 'claim-form',
    "description": 'Blueprint for Medical Claim form CMS 1500',
    "type": 'DOCUMENT',
    "stage": 'LIVE',
    "schema_path": 'data/blueprint/claims_form.json'
}
with open(blueprint['schema_path']) as f:
    blueprint_schema = json.load(f)
    blueprint_arn = helper_functions.create_or_update_blueprint(
        bda_client, 
        blueprint['name'], 
        blueprint['description'], 
        blueprint['type'],
        blueprint['stage'],
        blueprint_schema
    )
print(f"Created/Update Blueprint, blueprint_arn={blueprint_arn}")
blueprint = bda_client.get_blueprint(
    blueprintArn=blueprint_arn,
    blueprintStage='LIVE'
)
schema = json.loads(blueprint['blueprint']['schema'])
helper_functions.show_popup_link('View Blueprint', json.dumps(schema, indent=2), "blueprint_schema")

In [ ]:
response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': document_s3_uri
    },
    outputConfiguration={
        's3Uri': bda_s3_output_location
    },
    dataAutomationProfileArn = f'arn:aws:bedrock:{region_name}:{account_id}:data-automation-profile/us.data-automation-v1',
    blueprints=[
        {
            'blueprintArn': blueprint_arn
        }
    ]
)

invocationArn = response['invocationArn']
print(f'Invoked data automation job with invocation arn {invocationArn}')

In [ ]:
status_response = helper_functions.wait_for_completion(
            client=bda_client,
            get_status_function=bda_runtime_client.get_data_automation_status,
            status_kwargs={'invocationArn': invocationArn},
            completion_states=['Success'],
            error_states=['ClientError', 'ServiceError'],
            status_path_in_response='status',
            max_iterations=15,
            delay=30
)
if status_response['status'] == 'Success':
    job_metadata_s3_location = status_response['outputConfiguration']['s3Uri']
else:
    raise Exception(f'Invocation Job Error, error_type={status_response["error_type"]},error_message={status_response["error_message"]}')

### Retrieve job metadata
The results of the file processing are stored in the S3 bucket configured earlier. Let's retrieve and explore the job metadata response that BDA produces in the configured S3 output bucket.

The job metadata contains the job details including job_id, the job_status and the identified modality for the job. It also contains the output information with an S3 location for the results.

In [ ]:
job_metadata = json.loads(helper_functions.read_s3_object(job_metadata_s3_location))
job_id = job_metadata['job_id']
pd.set_option('display.max_colwidth', None)
job_status = pd.DataFrame({
    'job_id': [job_metadata['job_id']],
    'job_status': [job_metadata['job_status']],
    'semantic_modality': [job_metadata['semantic_modality']]
}).T
job_metadata_table = pd.DataFrame(job_metadata['output_metadata'][0]['segment_metadata']).fillna('').T
job_metadata_table.index.name='Segment Index'
job_metadata_json = JSON(job_metadata, root="job_metadata", expanded=True)
# Display the widget
display_functions.display_multiple(
    [display_functions.get_view(job_status), display_functions.get_view(job_metadata_table), display_functions.get_view(job_metadata_json)], 
    ["Job Status", "Output Info", "Metadata (JSON)"])

### View Segments and Matched Blueprints
As we can see in the job metadata, BDA creates a segment section each for each individual document that it has identified in the file. Each segment section has details on the matched blueprint and the results of the extraction. For each segment, BDA also outputs the page indices (one or more) from the original file.

We can now get the custom output corresponding to each segment and look at the insights that BDA custom output produces.

In [ ]:
asset_id = 0
segments_metadata = next(item["segment_metadata"]
                                for item in job_metadata["output_metadata"] 
                                if item['asset_id'] == asset_id)

custom_outputs = [json.loads(helper_functions.read_s3_object(segment_metadata.get('custom_output_path'))) if segment_metadata.get('custom_output_status') == 'MATCH' else None for segment_metadata in segments_metadata]

### View Custom output summary

In [ ]:
custom_outputs_json = JSON(custom_outputs, root="custom_outputs", expanded=False)
custom_outputs_table = pd.DataFrame(helper_functions.get_summaries(custom_outputs)).fillna('')

display_functions.display_multiple(
    [
        display_functions.get_view(custom_outputs_table.style.hide(axis='index')),
        display_functions.get_view(custom_outputs_json)
    ], 
    ["Table View", "Raw JSON"])

### Extract Inference Results and Store in S3

In [ ]:
inference_result = custom_outputs[0]['inference_result']
JSON(inference_result)

In [ ]:
agent_input_bucket, agent_input_key = helper_functions.get_bucket_and_key(agent_review_s3_input_location)
s3_client.put_object(
    Bucket=agent_input_bucket,
    Key=f'{agent_input_key}/{job_id}.json',
    Body=json.dumps(inference_result)
)

### Invoke Agent for Claim Verification

In [ ]:
inputText=f"Review the claim using claim form data in S3 URI {agent_review_s3_input_location}/{job_id}.json"
agent_answer = bedrock_utils.invoke_agent_helper(bedrock_agent_runtime_client=bedrock_agent_runtime, 
                    query=inputText, session_id=str(uuid.uuid4()),
                    agent_id=agent_id, alias_id=agent_alias_id,enable_trace=True)

In [ ]:
Markdown(agent_answer.replace('$', r'\$'))

## Clean Up

Let's delete the local copies of downloaded sample files and the file we uploaded to s3 input directory, as well as the generated job output files.


In [ ]:
# Delete S3 File
s3_client.delete_object(Bucket=target_s3_bucket, Key=target_s3_key)

# Delete local file
if os.path.exists(local_documents_path):
    shutil.rmtree(local_documents_path)

# Delete bda workshop location that we created our contents under in S3
#!aws s3 rm {bda_workshop_s3_location} --recursive